## Train and evaluate model using Scikit-learn and Keras wrapper 

In [7]:
import tensorflow as tf
from aMNWtModel import AMNWtModel


seed = 10
# np.random.seed(seed=seed)  
tf.random.set_seed(seed)
# dataset_file = "./Dataset/IJN1463_EXP_UB_Anne.npz"
# objective=['BIOMASS_KT2440_WT3']
# dataset_file = "./Dataset/IJN1463_10_UB.npz"
# objective=['BIOMASS_KT2440_WT3']
dataset_file = "./Dataset/e_coli_core_UB_100.npz"
objective=['BIOMASS_Ecoli_core_w_GAM']
# dataset_file = "./Dataset/biolog_iML1515_EXP_UB_Anne.npz"
# objective=['BIOMASS_Ec_iML1515_core_75p37M']

print("---------------------------------------- model ----------------------------------------")
model = AMNWtModel(dataset_file=dataset_file, 
                   objective=objective,
                   timestep=4,
                #    n_hidden=1, 
                   hidden_dim=50,
                   epochs=50, 
                   verbose=True,
                   batch_size=7)
model.printout()

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler 
from tools import MaxScaler
scaler= MaxScaler()
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocess(scaler)
model.preprocessing_for_specific_model()


# epoch =1
# batch_size = 7
epoch = 200
batch_size = 7


---------------------------------------- model ----------------------------------------
number of metabolites:  72
filtered measurements size:  1
dataset file: ./Dataset/e_coli_core_UB_100.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (100, 20) (100, 1)
training epochs: 50
training regression: True
training batch size: 7
training validation iter: 0
training early stopping: False


In [8]:
# cross validation
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold, cross_validate
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

estimator= KerasRegressor(build_fn=model.build_model, 
                          epochs=epoch, 
                          batch_size=batch_size, 
                          verbose=0)

scoring = {"loss_constraint":make_scorer(model.loss_constraint),
           "mse":make_scorer(model.mse),
           "R2":make_scorer(model.R2),
           }

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
fit_params = {'callbacks': [callback]}
fit_params = {}

kfold= KFold(n_splits=5,shuffle=True, random_state=seed)

results=cross_validate(estimator, 
                       model.X_train, 
                       model.Y_train, 
                       cv=kfold, 
                       n_jobs=5, 
                       scoring=scoring, 
                       fit_params=fit_params,
                       return_train_score=True)
results

{'fit_time': array([24.74944735, 24.23258448, 22.84510255, 22.83261538, 23.18954849]),
 'score_time': array([0.32391357, 0.36620688, 0.45799685, 0.67416024, 1.00572181]),
 'test_loss_constraint': array([0.00360305, 0.00417085, 0.00390249, 0.00492904, 0.00463018]),
 'train_loss_constraint': array([0.00359049, 0.00436789, 0.00445072, 0.00401283, 0.00453333]),
 'test_mse': array([0.00301429, 0.00328702, 0.00303203, 0.00384135, 0.00365976]),
 'train_mse': array([0.00292494, 0.003427  , 0.00352884, 0.00312779, 0.00361516]),
 'test_R2': array([0.92848833, 0.95878018, 0.9404594 , 0.97287722, 0.95699436]),
 'train_R2': array([0.94173666, 0.96305744, 0.95872752, 0.96707112, 0.94591472])}

In [9]:
AMNWt_model = model.build_model()
history = AMNWt_model.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model.predict(model.X_test)))

R2 : 0.951239231933068
Q2 : 0.8899012071581155


## Search for hyperparameters

In [10]:
from sklearn.model_selection import RandomizedSearchCV

estimator= KerasRegressor(build_fn=model.build_model, 
                          epochs=epoch, 
                          batch_size=batch_size, 
                          verbose=0)

distributions = dict(batch_size=[7,20],
                     nb_epoch=[2,100],
                    #  hidden_dim=[1,2],
                     )

scoring = {"loss_constraint":make_scorer(model.loss_constraint),
           "mse":make_scorer(model.mse),
           "R2":make_scorer(model.R2),
           }

clf = RandomizedSearchCV(estimator, distributions, random_state=0)
search = clf.fit(model.X_test, model.Y_test)

/home/agiralt/anaconda3/envs/AMN/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


## Save and load the model

In [11]:
import tensorflow as tf
from aMNWtModel import AMNWtModel


seed = 10
tf.random.set_seed(seed)
dataset_file = "./Dataset/e_coli_core_UB_100.npz"
objective=['BIOMASS_Ecoli_core_w_GAM']

# Dataset plus model structure
print("---------------------------------------- model ----------------------------------------")
model = AMNWtModel(dataset_file=dataset_file, 
                   objective=objective,
                   timestep=4,
                   hidden_dim=50,
                   epochs=50, 
                   verbose=True,
                   batch_size=7)
model.printout()

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler 
from tools import MaxScaler
scaler= MaxScaler()
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocess(scaler)
model.preprocessing_for_specific_model()

batch_size = 7

# Construct and train an AMNWt model
AMNWt_model = model.build_model()
history = AMNWt_model.fit(model.X_train, model.Y_train, epochs=200, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model.predict(model.X_test)))

---------------------------------------- model ----------------------------------------
number of metabolites:  72
filtered measurements size:  1
dataset file: ./Dataset/e_coli_core_UB_100.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (100, 20) (100, 1)
training epochs: 50
training regression: True
training batch size: 7
training validation iter: 0
training early stopping: False
R2 : 0.951239231933068
Q2 : 0.8899012071581155


In [12]:
from aMNWtModel import RNNCell
from tools import my_mse

seed = 10
tf.random.set_seed(seed)

# Recreate new model from config file, compile and train it. First test on config.
config = AMNWt_model.get_config()
AMNWt_model_= tf.keras.Model.from_config(config, custom_objects={"RNNCell":RNNCell})
AMNWt_model_.compile(loss=my_mse,optimizer='adam',metrics=[my_mse])
history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=200, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.951239231933068
Q2 : 0.8899012071581155


In [13]:
# Save the train AMNWt_model
model_file = "Models/test_saving_model.keras"
tf.keras.models.save_model(AMNWt_model,model_file, overwrite=True, save_format=None, save_traces=True)

In [14]:
from tools import my_mse
from aMNWtModel import RNNCell

AMNWt_model_ = tf.keras.models.load_model(model_file, custom_objects={"RNNCell":RNNCell,
                                                                               "my_mse":my_mse})

history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=200, batch_size=batch_size, verbose=0)
print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9819045023399564
Q2 : 0.9144268791917869
